In [1]:
"""
Check for Wyckoff bond.
"""
from multipie.util.util_binary import BinaryManager
from multipie.util.util import str_to_sympy
from multipie.util.util_wyckoff import find_wyckoff_bond, find_vector, shift_bond
from qtdraw import QtDraw
from multipie.core.multipie_info import __top_dir__
BIN_DIR = __top_dir__ + "others/tools/binary_data"
info = BinaryManager("info", verbose=True, topdir=BIN_DIR)
qtdraw = QtDraw()
group = BinaryManager("group", topdir=BIN_DIR)

load binary from '/Users/hk/Library/CloudStorage/Dropbox/working/GitHub/MultiPie2/others/tools/binary_data/info.pkl'.
"""
* Information of all groups.
- "id_set" (str): (dict) info. of group's IDs.
  - "PG/SG/MPG/MSG" (str):  (dict) ID set.
    - PG: all/crystal/complex/irrep.
    - SG: all/crystal/PG.
    - MPG: all/crystal/PG/type.
    - MSG: all/crystal/PG/SG/MPG/type.
- "tag" (str): (dict) from id to tag.
- "id" (str):  (dict) from tag to id.
- "character" (str): (dict) info. of character.
  - "alias" (str): (dict) alias.
    - "wp/wm" (str): (sympy) definition of wp and wm, exp(2 pi i/3) or exp(-2 pi i/3).
  - "compatibility_relation" (str): (dict) compatibility relation.
    - (PG_tag1,PG_tag2) (str,str): ([(str,str)]) list of compatibility relation, (irrep1,irrep2).
- "root_cluster" (str): (dict) info. of root cluster.
  - "point_group" (str): ([str]) list of PG_tag for root cluster.
  - "rep_site_c" (str): (ndarray(3,sympy)) representative site (fractional for cubic, r=1; = car

In [2]:
tag = "D2^5"
gp = group[info["id"][tag]]
wyckoff_site = gp["wyckoff"]["site"]
wyckoff_bond = gp["wyckoff"]["bond"]
pset = gp["symmetry_operation"].get("plus_set", None)
if pset is None:
    so = gp["symmetry_operation"]["fractional"]
else:
    so = gp["symmetry_operation"]["fractional_primitive"]
site_wp = list(wyckoff_site.keys())
bond_wp = list(wyckoff_bond.keys())
print("wyckoff site =", site_wp)
print("wyckoff bond =", bond_wp)
if pset is not None:
    print("plus set =", pset.tolist())
print()

bond = str_to_sympy("[-1,-1,-1,4/3,1/3,0]")
#b_wp = "4a@4b"
#bond = wyckoff_bond[b_wp]["reference"][0]

print("given bond =", bond.astype(float).tolist())
b_wp, all_bond = find_wyckoff_bond(gp, bond)
print("wp =", b_wp)
wyckoff_bond_wp = wyckoff_bond[b_wp]["conventional"]
print("wyckoff bond (no pset) =", wyckoff_bond_wp.tolist())

if pset is not None:
    bond = shift_bond(bond)
idx = find_vector(bond, all_bond)
qtdraw.set_crystal(gp["info"].crystal)
qtdraw.clear_data()
n = len(wyckoff_bond_wp)
for no, b in enumerate(all_bond):
    v, c = b[0:3], b[3:6]
    cl = "strawberry" if idx==no else "aqua"
    qtdraw.add_bond(position=c, direction=0.2*v, color=cl, label=f"s{no%n+1:02d}", name=f"p{no//n+1}")

wyckoff site = ['4a', '4b', '8c']
wyckoff bond = ['4a@4a', '4b@4a', '8c@4a', '4a@4b', '4b@4b', '8c@4b', '8a@8c']
plus set = [[0, 0, 0], [1/2, 1/2, 0]]

given bond = [-1.0, -1.0, -1.0, 1.3333333333333333, 0.3333333333333333, 0.0]
wp = 8a@8c
wyckoff bond (no pset) = [[X, Y, Z, x, y, z], [-X, -Y, Z, -x, -y, z + 1/2], [-X, Y, -Z, -x, y, 1/2 - z], [X, -Y, -Z, x, -y, -z]]
